# covalent_api

> Endpoints for the `covalent` api: https://www.covalenthq.com/docs/api/

In [ ]:
#|default_exp covalent_api

In [ ]:
#| hide
from nbdev.showdoc import *

**Functionality of `Covalent_Api` class:**


`get_historical_balances`

Used to fetch the historical native, fungible (ERC20), and non-fungible (ERC721 & ERC1155) tokens held by an address at a given block height or date. Response includes daily prices and other metadata.
https://www.covalenthq.com/docs/api/balances/get-historical-token-balances-for-address/

`get_token_holders`

Get token holders as of any block height (v2).
Commonly used to get a list of all the token holders for a specified ERC20 or ERC721 token. Returns historic token holders when block-height is set (defaults to latest). Useful for building pie charts of token holders.
Note: This gets the holders by percentage (i.e. largest holdest to smallest)
https://www.covalenthq.com/docs/api/balances/get-token-holders-as-of-any-block-height-v2/


`get_holders_portfolios`

Wrapper around `get_historical_balances` to update a list of holders with their portfolios.

**Other functionality of notebook:**

`union_top_n`

Computes all the coins in the top n portfolios of a token (i.e. union of all portfolios)

`intersection_count`

 For each token in the union, count how many holders have that token in their portfolio. Returns an `intersection_dict`.

`Address_Holder_Data`

 Wrapper to compute `intersection_dict` (i.e. return value from `intersection_count`) and intermediate values needed to compute it e.g. `union`.

`contract_name_if_k_holders`

Finds contracts with exactly k holders in `intersection_dict`

`contract_name_if_more_than_k_holders`

Finds contracts with more than k holders in `intersection_dict`

`contract_address_to_holders`

Finds the addresses in a collection that hold a specified contract address.



In [ ]:
#| export

import json
import requests
from requests.auth import HTTPBasicAuth
from requests.exceptions import HTTPError, RequestException
import time
from fastcore.basics import *
from fastcore.test import *

In [ ]:
#| export

import logging

# Set up basic logging configuration
logging.basicConfig(level=logging.WARNING, format='%(message)s')

logger = logging.getLogger(__name__)

In [ ]:
#| export

#The general API is: 
#                       def some_function(self,args):
#                            url = ... #define the url using args
#                            return self.get_items(url)

#rather than writing self.get_items(url) every time, we can use a decorator to do this for us.

def url_decorator(func):
    def wrapper(self, *args, **kwargs):
        url = func(self, *args, **kwargs)
        return self.get_items(url)
    return wrapper

class Covalent_Api:
    """This class is used to interact with the Covalent API.
    """

    def __init__(self, api_key='cqt_rQ37X8PXHqGxkycgdXHJtkFhfwyP'):
        self.api_key = api_key


    def get_items(self, url, retries=3, delay=1):
        """Given a url, get the items from the API.
        Inputs:
            `url`, a string, the url to get the items from.
            `retries`, an integer, the number of times to retry the request.
            `delay`, an integer, the number of seconds to wait between retries.

        Outputs:
            `items`, a list of dictionaries, each dictionary containing information about a token/address/etc
        """
        headers = {"accept": "application/json"}
        basic = HTTPBasicAuth(f'{self.api_key}', '')

        #TODO: e.g. if it is a mevbot with heaps of transactions this might fail: perhaps needs a timer.
        while retries > 0:
            try:
                response = requests.get(url, headers=headers, auth=basic)
                response.raise_for_status()  # Check for HTTP errors
                data = response.json()

                # Access 'data' field
                data = data.get('data')
                if data is None:
                    return None

                # Check if data is a list
                if type(data) is list:
                    return data

                # Access 'items' field
                return data.get('items')
            
            except HTTPError as e:
                if e.response.status_code == 429:  # HTTP Status Code for 'Too Many Requests'
                    time.sleep(delay)  # Adjust wait time
                    retries -= 1
                else:
                    return None

            except RequestException as e:
                return None


    @url_decorator
    def get_historical_balances(self,chainName:str,walletAddress:str,date:str, quote_currency="USD")->list:
        """
            API docs: https://www.covalenthq.com/docs/api/balances/get-historical-token-balances-for-address/
            Given a wallet address, get the historical balances for that wallet address on the specified chain and date.
            Inputs:
                `chainName`, e.g. 'eth-mainnet'
                `walletAddress`, e.g. '0x7364a0f792e073814B426c918bf72792575b6c18'
                `date`, e.g. '2021-01-01'.
                `quote_currency`, e.g. 'USD'
            Outputs:
                `items`, a list of dictionaries, each dictionary containing the balance of a token at a given date along with some additional metadata.
            Note:
                While the core function generates a URL, the applied @url_decorator 
                modifies the return behavior to fetch items using that URL.
        """
        assert type(date) is str, "date must be a string"

        url = f"https://api.covalenthq.com/v1/{chainName}/address/{walletAddress}/historical_balances/?quote-currency={quote_currency}&date={date}"

        return url 
    
    @staticmethod
    def print_balance(items):
        "Helper function to print the balance out, i.e. the object returned by `get_historical_balances`"

        for item in items:
            print(f"{item['contract_name']} balance: {item['balance']}\n")
    
    @url_decorator
    def get_token_holders(self, chainName: str, tokenAddress: str, block_height=None, page_size=100, page_number=None) -> list:
        """
        API docs: https://www.covalenthq.com/docs/api/balances/get-token-holders-as-of-any-block-height-v2/
        Fetches the token holders for a specific token on a given chain.
        Note: There is a possible `block-height` parameter, which we omit for now (see the API docs to clarify)
        
        Inputs:
            `chainName`: The chain name e.g. 'eth-mainnet'.
            `tokenAddress`: The token's address.
            `block_height`: Ending block to define a block range.
            `page_size`: Number of items per page. Supported values are 100 and 1000.
            `page_number`: 0-indexed page number to begin pagination.
        Outputs:
            `items`, a list of dictionaries, each dictionary containing information about a token holder.

        Note: This gets the holders by percentage (i.e. largest holdest to smallest)

        Note:
        While the core function generates a URL, the applied @url_decorator 
        modifies the return behavior to fetch items using that URL.
        """

        base_url = f"https://api.covalenthq.com/v1/{chainName}/tokens/{tokenAddress}/token_holders_v2/?"

        url = "https://api.covalenthq.com/v1/eth-mainnet/tokens/{tokenAddress}/token_holders_v2/?"

        
        if block_height is not None:
            base_url += f"&block-height={block_height}"
        if page_size is not None:
            base_url += f"&page-size={page_size}"
        if page_number is not None:
            base_url += f"&page-number={page_number}"

        return base_url

    def get_holders_portfolios(self,n_lst,chainName:str,date:str,quote_currency="USD",log_output=False):
        """Input: 
                `n_lst`: a list of dictionaries containing the top holders of a token. Dictionary must contain the key 'address'.
                 See get_historical_balances for description of other inputs.
           Output: the same list of dictionaries with the portfolio added.
        """

        # Setting up logging level based on the log_output value
        if log_output:
            logger.setLevel(logging.INFO)
        else:
            logger.setLevel(logging.WARNING)

        #loop over the list of dictionaries (holders of the token)
        for _holder in n_lst: #this loop should be parallelized, but ok for now provided it isn't too big.

            _address = _holder['address']
            _items = self.get_historical_balances(chainName=chainName,walletAddress=_address, quote_currency=quote_currency, date=date)
            #_items is a list of dictionaries

            #get the sum of the portfolio

            #TODO: Make this more efficient: possibly convert to numpy array or dataframe or something and then sum
            none_to_zero = lambda x: 0 if x is None else x
            portfolio_sum = sum([none_to_zero(holding['quote']) for holding in _items])
            
            _holder['portfolio']=_items #update the _holder with the whole portfolio
            _holder['portfolio_sum']=portfolio_sum #update the _holder with the total value of the portfolio

            logger.info(f'got {_address} portfolio')  # Logging statement in place of print
            
        return n_lst
    
    #It seems that we already have this data via `get_historical_balances` throughts ['items']['quote'] #usd value
    # @url_decorator
    # def get_prices(self,chainName:str,address:str,quote_currency="USD",dates=None):
    #     """Input: 
    #             `chainName`: e.g. 'eth-mainnet'
    #             `address`: e.g. '0x7364a0f792e073814B426c918bf72792575b6c18'
    #             `quote_currency`: e.g. 'USD'
    #             `dates`: e.g. ['2021-01-01','2021-01-10'], i.e. from,to
    #        Output: 
    #     """

    #     if len(dates)==1:
    #         dates.append(dates[0]) #if only one date is specified, then we get the price for that date only.

    #     url = f"https://api.covalenthq.com/v1/pricing/historical_by_addresses_v2/{chainName}/{quote_currency}/{address}/?from={dates[0]}&to={dates[1]}"
    #     return url
    
    @staticmethod
    def print_prices(items):
        "Helper function to print the prices out i.e. `items` as returned by `get_prices`"
        print(f"Printing out prices of: {items[0]['contract_name']}")
        for k in items[0]['prices']:
            print(f"On {k['date']}, the price was {k['price']}, and the `pretty_price` was {k['pretty_price']}")
            


How to use:

In [ ]:
###Setup: hyperparameters
cov_api = Covalent_Api() #might need to pass an API key (different to default -- see __init__ of class)
tokenAddress = '0x72e4f9F808C49A2a61dE9C5896298920Dc4EEEa9' #an erc20 token address (hpbitcoin)
chainName='eth-mainnet'
###

**How to use `get_token_holders`**: it is used to get the top token holders of a token.

In [ ]:
#get the top 100 holders of the given address
token_holders_hpbitcoin = cov_api.get_token_holders(chainName=chainName,tokenAddress=tokenAddress,page_size=100,page_number=0)
assert len(token_holders_hpbitcoin)==100
top_10_holders_hpbitcoin = token_holders_hpbitcoin[:10]
assert len(top_10_holders_hpbitcoin)==10

In [ ]:
#TODO: more tests here of `get_token_holders` here

**How to use `get_historical_balances`**:

#NOTE: Basically a helper function to compute `get_holders_portfolios`

In [ ]:
#New args required:
walletAddress = top_10_holders_hpbitcoin[3] #a wallet address
date = '2023-08-30' #a date

#Get the historical balances for a given wallet address on a given chain and date
portfolio = cov_api.get_historical_balances(chainName=chainName,walletAddress=walletAddress,date=date)

In [ ]:
#TODO: tests of `get_historical_balances` here

**How to use `get_holders_portfolios`**: requires a list of holders, e.g. output from `get_token_holders`. Basically it will update the token holders with their portfolios on the provided date (so a wrapper for `get_historical_balances`):

In [ ]:
#now update token holders info by getting their whole portfolios on the given `date`. We take the list output from `get_token_holders` and update it with the portfolio info.
date = '2023-08-30'
top_10_holders_hpbitcoin = cov_api.get_holders_portfolios(n_lst=top_10_holders_hpbitcoin,chainName=chainName,date=date)

#Can now also get information like: what was the total portfolio value of the 4th largest holder of hpbitcoin on the given date?
print(f"Total portfolio value of the 4th largest holder of hpbitcoin on {date} was: ${top_10_holders_hpbitcoin[3]['portfolio_sum']} USD")
#top_10_holders_hpbitcoin[3]['portfolio'] gives the whole portfolio of the 4th largest holder of hpbitcoin on the given date.


Total portfolio value of the 4th largest holder of hpbitcoin on 2023-08-30 was: $1880739.477863531 USD


In [ ]:
#| export

def union_top_n(top_n:list)->list:
    """Computes all the coins in the top n portfolios of a token (i.e. union of all portfolios)    
    """

    top_n = list(set(
        (item['contract_name'],item['contract_address'])
        for entry in top_n #Basically: for each address in top_10_nickcage get all the tokens in their portfolio
        for item in entry['portfolio']
                    ))
    return top_n

**How to use `union_top_n`**:

In [ ]:
union_top_10_holders_hpbitcoin = union_top_n(top_10_holders_hpbitcoin)#i.e. all the tokens ("union") help by the top 10 hpbitcoin holders

In [ ]:
#| export

def intersection_count(top_n:list, union_lst:list) -> dict:
    """
    For each token in the union, count how many holders have that token in their portfolio.
    Inputs:
        top_n: list of dictionaries, each dictionary is a holder
        union_lst: list of tokens, each token is a string
    Output:
        intersect_dict: dictionary, keys are tokens, values are integers
    """

    # Create a dictionary with default value as 0
    intersect_dict = {(token_name, token_address): 0 for token_name, token_address in union_lst}

    # Pre-compute contract addresses for each holder
    holder_portfolios = {}
    for holder in top_n:
        holder_portfolios[holder['address']] = set(item['contract_address'] for item in holder['portfolio'])

    # Update the intersection count
    for token_name, token_address in union_lst:
        for holder_address, contracts in holder_portfolios.items():
            if token_address in contracts:
                intersect_dict[(token_name, token_address)] += 1

    return intersect_dict



**How to use `intersection_count`**:


In [ ]:
intersect_dict_top_10_hpbitcoin = intersection_count(top_10_holders_hpbitcoin,union_top_10_holders_hpbitcoin)

And we can verify the following **tests**:
- that all holders amongst the top 10 of `bitcoin` hold `bitcoin` respectively (this must be vacuously true).
- that the counts are all larger than 1 (at least one address must own the token for it to have appeared in the union)

In [ ]:
#names of the tokens
hpbitcoin_name = top_10_holders_hpbitcoin[0]['contract_name']
#addresses
hpbitcoin_address = top_10_holders_hpbitcoin[0]['contract_address']

#check that `nickcage` and `hpbitcoin` occur exactly 10 times in the top 10
test_eq(intersect_dict_top_10_hpbitcoin[(hpbitcoin_name,hpbitcoin_address)],10)

#Check that each token appears at least once 
for k in intersect_dict_top_10_hpbitcoin: assert intersect_dict_top_10_hpbitcoin[k]>=1

In [ ]:
#| export

def contract_name_if_k_holders(intersect_dict,k):
    "Get the contracts that have exactly k holders"
    if k>len(intersect_dict):
        raise ValueError("k is greater than length of intersect_dict")
    return [contract_name for contract_name in intersect_dict.keys() if intersect_dict[contract_name] == k]

def contract_name_if_more_than_k_holders(intersect_dict,k):
    "Get the contracts that have more than k holders"
    if k>len(intersect_dict):
        raise ValueError("k is greater than length of intersect_dict")
    return [contract_name for contract_name in intersect_dict.keys() if intersect_dict[contract_name] > k]

**How to use `contract_name_if_k_holders` and `contract_name_if_more_than_k_holders`**:

In [ ]:
hpbitcoin_name_if_k_holders = contract_name_if_k_holders(intersect_dict_top_10_hpbitcoin,k=10)
hpbitcoin_name_if_more_than_k_holders = contract_name_if_more_than_k_holders(intersect_dict_top_10_hpbitcoin,k=8)

print(f"hpbitcoin_name_if_k_holders: {hpbitcoin_name_if_k_holders}")
print(f"hpbitcoin_name_if_more_than_k_holders: {hpbitcoin_name_if_more_than_k_holders}")

hpbitcoin_name_if_k_holders: [('Ether', '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'), ('HarryPotterObamaSonic10Inu', '0x72e4f9f808c49a2a61de9c5896298920dc4eeea9'), ('ETHEREUM2.0', '0xea498670e8de236c17d75c0bd09dd4b1b6f39eb2')]
hpbitcoin_name_if_more_than_k_holders: [('EpsteinClintonQanonPizzaGate10Inu', '0x682a21d52451bb93e4bda3c557e46e0016d0edb0'), ('Bald', '0x02c9aaa47de7be1d08f551a59016a562ad11fb5f'), ('Fino', '0x96a3b1227fdbe7342d1358ae4ed87adc4584d9cf'), ('HarryPotterObamaPacMan8Inu', '0x07e0edf8ce600fb51d44f51e3348d77d67f298ae'), ('XDOGE', '0xd2b274cfbf9534f56b59ad0fb7e645e0354f4941'), ('DFI.money2.0', '0xbc581118802bec35e755f0a85f62af6736a2c4a1'), ('Bullet Game V2', '0x9333ac938a18decb74425bd9bdd6cca3c5998875'), ('Friend Tech', '0x88c6aefe66fc619d073d3560bb5dc4d8a38f3e8c'), ('Ether', '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'), ('Friend Tech', '0x2cf82a3ad32aa580110278022a6493b631044bef'), ('HarryPotterObamaSonic10Inu', '0x72e4f9f808c49a2a61de9c5896298920dc4eeea9'), ('Fuck Ba

In [ ]:
#| export

def contract_address_to_holders(n_holders,contract_address):
    "Given an address, get the holders that hold that address"
    lst=[]
    for _holder in n_holders:
        for _token in _holder['portfolio']:
            if _token['contract_address'] == contract_address:
                lst.append(_holder)

    return lst

**How to use `contract_address_to_holders`**:

In [ ]:
xdoge_address = "0xd2b274cfbf9534f56b59ad0fb7e645e0354f4941" #can observe that this is in hpbitcoin_name_if_more_than_k_holders with k=8 (see above)

#the holders of hpbitcoin who also hold xdoge
hpbitcoin_xdoge_holders = contract_address_to_holders(top_10_holders_hpbitcoin,xdoge_address)
print(f"The holders of hpbitcoin who also hold xdoge are: {hpbitcoin_xdoge_holders}")
print(f"There are {len(hpbitcoin_xdoge_holders)} such holders")


The holders of hpbitcoin who also hold xdoge are: [{'contract_decimals': 8, 'contract_name': 'HarryPotterObamaSonic10Inu', 'contract_ticker_symbol': 'BITCOIN', 'contract_address': '0x72e4f9f808c49a2a61de9c5896298920dc4eeea9', 'supports_erc': ['erc20'], 'logo_url': 'https://logos.covalenthq.com/tokens/1/0x72e4f9f808c49a2a61de9c5896298920dc4eeea9.png', 'address': '0x51205a732f40e57d58c7823430e8168c4a0e685b', 'balance': '2600000100000000', 'total_supply': '100000000000000000', 'block_height': 18032186, 'portfolio': [{'contract_decimals': 8, 'contract_name': 'HarryPotterObamaSonic10Inu', 'contract_ticker_symbol': 'BITCOIN', 'contract_address': '0x72e4f9f808c49a2a61de9c5896298920dc4eeea9', 'supports_erc': ['erc20'], 'logo_url': 'https://logos.covalenthq.com/tokens/1/0x72e4f9f808c49a2a61de9c5896298920dc4eeea9.png', 'block_height': 18023703, 'last_transferred_block_height': 17773276, 'last_transferred_at': '2023-07-25T23:07:23Z', 'native_token': False, 'type': 'cryptocurrency', 'is_spam': Fal

We can collect the `intersect_dict`, and other data through a simple wrapper:

In [ ]:
#| export

class Address_Holder_Data:

    def __init__(self, cov_api, tokenAddress, chainName,date,n,quote_currency="USD"): 
        store_attr()
        assert n <= 100, f"The value of n should be <= 100. The current value is {n}."
        
        self.intersect_dict_top_n = self.get_data()
        
    def get_data(self):
        self.token_holders = self.cov_api.get_token_holders(chainName=self.chainName, tokenAddress=self.tokenAddress, page_size=100, page_number=0)
        self.top_n_holders = [self.token_holders[i] for i in range(self.n)]
        self.top_n_holders = self.cov_api.get_holders_portfolios(n_lst=self.top_n_holders,chainName=self.chainName,quote_currency=self.quote_currency,date=self.date)
        self.union_top_n = union_top_n(self.top_n_holders)
        self.intersect_dict_top_n = intersection_count(top_n=self.top_n_holders, union_lst=self.union_top_n)

        return self.intersect_dict_top_n


**How to use `Address_Holder_Data`**:

In [ ]:
#Just a wrapper to get the `intersect_dict` (i.e. the data) for a given token address, chain name, date, and k. Note that
#to compute this we also need to get the top n holders of the token, and their portfolios, and the union of all the tokens in the top n portfolios.
n=10
hpbitcoin_holder_data = Address_Holder_Data(cov_api=cov_api,tokenAddress=tokenAddress,chainName=chainName,date=date,n=n)

#Check that it holds the following data:
assert len(hpbitcoin_holder_data.top_n_holders) is n #top k holders of hpbitcoin (`tokenAddress`)
hpbitcoin_holder_data.union_top_n #union of all the tokens in the top k portfolios
hpbitcoin_holder_data.intersect_dict_top_n

{('TOSHI', '0xa0e7d36bee881288f2701c09b00270aa2c731dc0'): 3,
 ('Axie Infinity', '0xaaf3385bf59e0dbbebd7c80a592bd05cc2bd8c74'): 1,
 ('X', '0x7e9f4e9edea00d1453491cfdc46b8299f4fd4ea5'): 1,
 ('Open DEX', '0x0015299a3790853a15a49ac559eaa04fba291d6c'): 1,
 ('HEXEx', '0xa10c9a47adea576f9568fdaf797a17cbfa23a9f1'): 3,
 ('HermioneRooseveltMortalKombat69inu',
  '0x2d9aa3dffc9d0cd05ed520f2804b1b9aa1b761f1'): 1,
 ('MouseWormV2', '0x7d3cc53a168876b6da644fd259c0305a9133a63d'): 8,
 ('Edelweiss', '0x8b0b0e6bc81bc0595fd59c014a1c5a5d6e70d730'): 1,
 ('Misty Inu', '0x814e8f8e35ac8bab913326364ca47bf8f6571442'): 1,
 ('GeGeCoin', '0x0843ea026f458ba1b7dd86b0b4f6434923dbd065'): 1,
 ('HEX', '0x2b591e99afe9f32eaa6214f7b7629768c40eeb39'): 3,
 ('Saitama Inu', '0x8b3192f5eebd8579568a2ed41e6feb402f93f73f'): 1,
 ('Bitlord', '0x781bd109834c534dc0f799afdf65e6eb5151b839'): 8,
 ('USBTC', '0x09573e7b80a68a197f358dc47328381bec110b13'): 1,
 ('Gigachad Kishu', '0x2b7184f2cba49bf7093297951598aea0f546f0be'): 1,
 ('Bloop Coin',

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()